In [3]:
import numpy as np
from pysr import PySRRegressor
from update_expressions import update_expressions, ExpressionUpdater

# 生成测试数据
np.random.seed(42)  # 设置随机种子以确保可重复性
n_samples = 1000

# 生成时间序列
t = np.linspace(0, 10, n_samples)

# 生成阻尼振荡器的数据
omega = 2.0  # 角频率
gamma = 0.1  # 阻尼系数

# 真实的位置函数（解析解）
x = np.exp(-gamma * t) * np.cos(np.sqrt(omega**2 - gamma**2) * t)
v = -gamma * x + np.exp(-gamma * t) * np.sin(np.sqrt(omega**2 - gamma**2) * t)

# 添加一些噪声
x_noisy = x + np.random.normal(0, 0.01, n_samples)
v_noisy = v + np.random.normal(0, 0.01, n_samples)

# 计算真实的加速度（用于训练）
a = -omega**2 * x - 2*gamma * v

# 准备训练数据
X = np.column_stack((x_noisy, v_noisy))
y = a

# 配置 PySR
model = PySRRegressor(
    niterations=20,  # 减少迭代次数以加快测试
    binary_operators=["+", "*", "-", "/"],
    unary_operators=["sin", "cos"],
    maxsize=10,
    populations=20,
    verbosity=1
)

print("开始训练模型...")
model.fit(X, y)
print("训练完成!\n")

# 显示前5个最佳表达式
print("\n按损失值排序的前5个表达式：")
print("=" * 50)
sorted_equations = model.equations_.sort_values('loss', ascending=True)
for i in range(min(5, len(sorted_equations))):
    expr = sorted_equations.iloc[i]
    print(f"\n第{i+1}名表达式:")
    print(f"SymPy形式: {expr['sympy_format']}")
    print(f"损失值: {expr['loss']:.6e}")
    print("-" * 30)

开始训练模型...


c:\anacoda\envs\pytorch\Lib\site-packages\pysr\sr.py:2766: UserWarning: Note: it looks like you are running in Jupyter. The progress bar will be turned off.
  warnings.warn(
[ Info: Started!
[ Info: Final population:
[ Info: Results saved to:


───────────────────────────────────────────────────────────────────────────────────────────────────
Complexity  Loss       Score      Equation
1           3.483e+00  1.594e+01  y = -0.084803
3           1.031e-02  2.911e+00  y = x₀ / -0.25059
5           1.025e-02  2.588e-03  y = -0.0073072 - (x₀ * 3.9899)
7           1.529e-03  9.515e-01  y = (x₀ * -3.9985) - (x₁ * 0.20259)
8           1.529e-03  3.602e-04  y = (x₀ * -3.9984) - sin(x₁ * 0.2032)
9           1.528e-03  3.318e-04  y = sin(sin(x₁ * -0.20383)) + (x₀ * -3.9984)
10          1.528e-03  2.909e-04  y = (x₀ * -3.9986) - sin((0.0048461 - x₁) * -0.20339)
───────────────────────────────────────────────────────────────────────────────────────────────────
训练完成!


按损失值排序的前5个表达式：

第1名表达式:
SymPy形式: x0*(-3.9985516) - sin((0.0048460974 - x1)*(-0.20339295))
损失值: 1.527625e-03
------------------------------

第2名表达式:
SymPy形式: x0*(-3.9984415) + sin(sin(x1*(-0.20382892)))
损失值: 1.528069e-03
------------------------------

第3名表达式:
SymPy形式: x0*(-3

In [4]:
# 测试 update_expressions 模块

# 1. 测试 numpy 版本
print("测试 numpy 版本:")
print("=" * 50)
update_expressions('specification_oscillator1_numpy.txt', model, top_n=3)
print("numpy 版本测试完成!\n")

# 2. 测试 body 版本
print("\n测试 body 版本:")
print("=" * 50)
update_expressions('specification_oscillator1_body.txt', model, top_n=5)
print("body 版本测试完成!")

测试 numpy 版本:
numpy 版本测试完成!


测试 body 版本:
body 版本测试完成!


In [5]:
# 详细测试 ExpressionUpdater 类

# 测试 numpy 版本的详细信息
updater_numpy = ExpressionUpdater('specification_oscillator1_numpy.txt')
print("Numpy 版本信息:")
print(f"变量映射: {updater_numpy.var_mapping}")
print(f"数学库前缀: {updater_numpy.math_prefix}\n")

# 获取处理后的表达式
processed_exprs = updater_numpy.process_expressions(model, top_n=2)
print("处理后的表达式:")
for i, expr in enumerate(processed_exprs, 1):
    print(f"\n表达式 {i}:")
    print(expr)
    print("-" * 30)

Numpy 版本信息:
变量映射: {'x0': 'x', 'x1': 'v'}
数学库前缀: jnp.

处理后的表达式:

表达式 1:
dv = x*(params[0]) - jnp.sin((params[1] - v)*(params[2]))
return dv
------------------------------

表达式 2:
dv = x*(params[0]) + jnp.sin(jnp.sin(v*(params[1])))
return dv
------------------------------
